#Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import layers

#Implement a Transformer block as a layer

In [ ]:
class TransformerBlock(layers.Layer) :
  def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1) :
    super().__init__()
    self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    self.ffn = keras.Sequential(
        [layers.Dense(ff_dim, activation='relu'),
         layers.Dense(embed_dim)]
    )
    self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = layers.Dropout(rate)
    self.dropout2 = layers.Dropout(rate)

  def call(self, inputs, training) :
    attn_output = self.att(inputs, inputs)
    attn_output = self.dropout1(attn_output, training=training)

    out1 = self.layernorm1(inputs + attn_output)
    
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout(ffn_output, training=training)
    return self.layernorm2(out1 + ffn_output)

# Implement embedding layer

Two seperate embedding layers, one for tokens, one for token index(positional)

In [ ]:
class TokenAndPositionalEmbedding(layers.Layer) :
  def __init__(self, maxlen, vocab_size, embed_dim) :
    sueper().__init__()
    self.token_emb = layers.Embedding(input_dim = vocab_size, output_dim=embed_dim)
    self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

  def call(self,x) :
    maxlen = tf.shape(x)[-1]
    positions = tf.range(start=0, limit=maxlen, delta=1)
    positions = self.pos_emb(positions)

    x= self.token_emb(x)
    return x+positions

# Download and prepare dataset

In [ ]:
vocab_size = 20000
maxlen = 200
(x_train, y_train), (x_val, y_val) = keras.dataset.imdb.load_data(num_words = vocab_size)

print(len(x_train), 'training sequences')
print(len(x_val), 'validation sequences')
x_train= keras.preprocessing.sequence.pad_sequence(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequence(x_val, maxlen=maxlen)

#Create classifier model using transformer layer

Transformer layer outputs one vector for each time step of our input seuqence.

In [ ]:
embed_dim = 32  ## Embedding size for each token
num_heads = 2   ## #attention heads
ff_dim = 32     ## hidden layer size in feed forward network inside transformer


intputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)

transformer_block = TransformerBlock(embed_dim, num_hedas, ff_dim)

x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = Dense(20, activation='relu')(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=ouputs)

#Train and Evaluate

In [ ]:
model = compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data = (x_val, y_val))